Ok so lets define a simple Parameter class which has a value which is stored as a numpy as array.
We also set a value property and setter
Then we define a `tie` function which take a parameter as an input and sets the its own parameter with the numpy array of the other one

In [75]:
import numpy as np

class Parameter(object):

    def __init__(self, value):
        self._value = np.array([value])
        self._untied_value = self._value
        self._tied=False
        self._tied_args=None
        self._tied_func=None
        
        
    @property
    def value(self):
        # if tied to a function evaluate it and return else just return
        # _value
        if self._tied_func is None :
            return self._value[0]
        else:
            return self._tied_func(*[arg for arg in self._tied_args])

    @property
    def tied(self):
        return self._tied
    
    @value.setter
    def value(self, value):
        self._value[0] = value
    
    def tie(self, *args):
        '''
        if first arg is a parameter tie values 
        if first arg is a function put the rest of the args in a list 
        so function can be evaluated at a later time
        '''
        self._tied=True
        if isinstance(args[0],Parameter):
            #lets keep a handle on origonal value
            self._untied_value=self._value 
            self._value = args[0]._value
        elif callable(args[0]):
            self._tied_func=args[0]
            self._tied_args=[arg for arg in args[1:]]
    
    def untie(self):
        '''
        removes the tied things
        '''
        self._value=self._untied_value
        self._tied=False
        self._tied_args=None
        self._tied_func=None

So lets make some parameters and tie one to the other and test if the *tied* parameter changes when you set the other one

In [76]:
pp = Parameter(5)
qq = Parameter(6)
print "original:", pp.value #output original value
pp.tie(qq)
print "tied:", pp.value #output tied value
qq.value = 10
print "updated:", pp.value #output changes value (hopefully)

original: 5
tied: 6
updated: 10


Currently astropy models use a _parameters property which houses a numpy array which can be handy so as to not break this lets build the array when its request it and then when it's set we distribute the values to the parameters (btw `_param_meterics` in astropy models will make the setting and getting nice as it houses the slices for each parameter)`

In [77]:
class Model(object): 
    def __init__(self):
        self.par1 = Parameter(1)
        self.par2 = Parameter(2)
        #this is here to save going though every attr to find the parameters
        self._param_attrs = [self.par1, self.par2]
        
    @property
    def _parameters(self):
        return np.array([p.value for p in self._param_attrs])
    
    @_parameters.setter
    def _parameters(self, value):
        for vv, par in zip(value, self._param_attrs):
            par.value = vv

Lets check this works as expected

In [78]:
mod=Model() 
print "original:", mod._parameters #output original values
mod._parameters = np.array([2, 3])#set new values
print "updated:", mod._parameters #hopefully output new values

original: [1 2]
updated: [2 3]


So to check the behavior we want still holds

In [85]:
mod2 = Model() #second model
mod.par1.value = 5 #reset parameter 1 of mod
print "original:", mod2._parameters #output original value for parameters
mod2.par1.tie(mod.par1) #tie mod2.par1 to mod.par1
print "tied:", mod2._parameters #output updated parameters
mod.par1.value = 10 #change mod.par1 this should change mod2.par1
print "updated:", mod2._parameters #output hopefully updated value
mod2.par1.untie()
print "untied:", mod2._parameters #hopefully back to the start

original: [1 2]
tied: [5 2]
updated: [10  2]
untied: [1 2]


We can also define a function which we can then evaluate when we want the value

In [86]:
mod3 = Model()
mod.par1.value = 5 #reset parameter 1 of mod
print "original:", mod3._parameters #output original value for parameters

def tiefunc(p1):
    return 2*p1.value

mod3.par1.tie(tiefunc,mod.par1) #tie mod2.par1 to mod.par1
print "tied:", mod3._parameters #output updated parameters
mod.par1.value = 10 #change mod.par1 this should change mod2.par1
print "updated:", mod3._parameters #output hopefully updated value
mod3.par1.untie()
print "untied:", mod3._parameters #hopefully back to the start


original: [1 2]
tied: [10  2]
updated: [20  2]
untied: [1 2]
